# 01 - Infrastructure Setup

This notebook sets up the Databricks infrastructure:
- Install custom SFTP data source package
- Configure storage credentials
- Set up Databricks secrets
- Verify connections

## 1. Install Custom SFTP Data Source Package

In [ ]:
# Install dependencies from requirements.txt
%pip install -r /Workspace/Repos/<your-repo>/databricks-sftp-data-source/requirements.txt

# Install the custom SFTP package
%pip install -e /Workspace/Repos/<your-repo>/databricks-sftp-data-source

dbutils.library.restartPython()

## 2. Import Libraries

In [ ]:
from pyspark.sql import SparkSession
from ingest import SFTPWriter, SFTPDataSource
import json

## 3. Configure Databricks Secrets

Store sensitive credentials in Databricks secrets. Run these commands in Databricks CLI:

```bash
# Create secret scope
databricks secrets create-scope --scope sftp-credentials

# Store source SFTP credentials
databricks secrets put --scope sftp-credentials --key source-host
databricks secrets put --scope sftp-credentials --key source-username
databricks secrets put --scope sftp-credentials --key source-private-key

# Store target SFTP credentials
databricks secrets put --scope sftp-credentials --key target-host
databricks secrets put --scope sftp-credentials --key target-username
databricks secrets put --scope sftp-credentials --key target-private-key
```

## 4. Upload SSH Private Key to Databricks

Upload the private key file to DBFS:

In [ ]:
# Create directory for SSH keys
dbutils.fs.mkdirs("/FileStore/ssh-keys")

# Upload private key using Databricks CLI:
# databricks fs cp ~/.ssh/sftp_key dbfs:/FileStore/ssh-keys/sftp_key

print("Upload SSH private key to: dbfs:/FileStore/ssh-keys/sftp_key")

## 5. Configure SFTP Connection Parameters

In [ ]:
# Source SFTP configuration
source_config = {
    "host": dbutils.secrets.get(scope="sftp-credentials", key="source-host"),
    "username": dbutils.secrets.get(scope="sftp-credentials", key="source-username"),
    "private_key_path": "/dbfs/FileStore/ssh-keys/sftp_key",
    "port": 22
}

# Target SFTP configuration
target_config = {
    "host": dbutils.secrets.get(scope="sftp-credentials", key="target-host"),
    "username": dbutils.secrets.get(scope="sftp-credentials", key="target-username"),
    "private_key_path": "/dbfs/FileStore/ssh-keys/sftp_key",
    "port": 22
}

print("SFTP configurations loaded")

## 6. Test Source SFTP Connection

In [ ]:
# Test connection to source SFTP
source_writer = SFTPDataSource.create_writer(source_config)

with source_writer.session():
    files = source_writer.list_files(".")
    print("Source SFTP files:")
    for f in files:
        print(f"  - {f}")

## 7. Test Target SFTP Connection

In [ ]:
# Test connection to target SFTP
target_writer = SFTPDataSource.create_writer(target_config)

with target_writer.session():
    files = target_writer.list_files(".")
    print("Target SFTP files:")
    for f in files:
        print(f"  - {f}")

## 8. Save Configuration to Catalog

In [ ]:
# Create catalog and schema for configuration
spark.sql("CREATE CATALOG IF NOT EXISTS sftp_demo")
spark.sql("CREATE SCHEMA IF NOT EXISTS sftp_demo.config")

# Store configuration (without sensitive data)
config_data = [
    ("source_host", source_config["host"]),
    ("source_username", source_config["username"]),
    ("target_host", target_config["host"]),
    ("target_username", target_config["username"]),
    ("ssh_key_path", "/dbfs/FileStore/ssh-keys/sftp_key")
]

config_df = spark.createDataFrame(config_data, ["key", "value"])
config_df.write.mode("overwrite").saveAsTable("sftp_demo.config.connection_params")

print("Configuration saved to sftp_demo.config.connection_params")

## 9. Verify Configuration

In [ ]:
# Display configuration
display(spark.table("sftp_demo.config.connection_params"))

## Summary

Infrastructure setup completed:
- ✓ Custom SFTP data source package installed
- ✓ Databricks secrets configured
- ✓ SSH keys uploaded to DBFS
- ✓ SFTP connections tested
- ✓ Configuration saved to Unity Catalog

Next step: Run notebook `02_uc_connection_setup.ipynb` to configure Unity Catalog connections